In [1]:
import GSOM
import numpy as np
import matplotlib.pyplot as plt
import time
import networkx as nx
import itertools
import cProfile
import pstats
%matplotlib qt

In [4]:
dataset = np.load("data/strawberry_peak.npy")
dataset = dataset * 1000
np.random.shuffle(dataset)

In [5]:
basis = GSOM.GSOM(dataset,0.94,0.10)

profile = cProfile.Profile()
profile.enable()
basis.train()
profile.disable()
ps = pstats.Stats(profile)
ps.strip_dirs()

ps.sort_stats("cumulative")
ps.print_stats()

Growth Threshold = 0.12375080743617506
Weight Change 1.5499546548744002e-07
Average error 0.05063204729994221
Max error 0.12257650671705068
Weights didn't change
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangle
Collapsing Triangl

In [6]:
data = np.array(basis.data)

x_data = data[:,[0]] / 1000
y_data = data[:,[1]] / 1000
print(basis.network)
plt.plot(x_data, y_data, '.', markerfacecolor='red')
basis.print()

basis.visualizeGraph()

plt.show()

Graph with 750 nodes and 755 edges
Triangles: 0.0


In [8]:
from scipy import spatial
import numba as nb
@nb.njit(fastmath=True)
def norm(l):
    s = 0.
    for i in range(l.shape[0]):
        s += l[i]**2
    return np.sqrt(s)
x = list(basis.network.nodes)
output_list = [i.array for i in x]


In [17]:
datapoint = np.random.rand(2).astype(np.float32)

print(f'Numpy Lin Alg:')
%timeit for y in output_list: np.linalg.norm(y-datapoint)



print(f'JIT Norm:')
%timeit for y in output_list: norm(y-datapoint)


tree = spatial.cKDTree(output_list)
print(f'KDTree Query:')
%timeit tree.query(datapoint,p=2,workers=4)





Numpy Lin Alg:
4.78 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
JIT Norm:
1.17 ms ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
KDTree Query:
38.1 µs ± 2.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
(123.92509819328679, 127)


In [19]:
print(tree.query(datapoint,k=2,p=2,workers=4))

(array([123.92509819, 123.92513956]), array([127, 409], dtype=int64))


: 

In [5]:
import osmnx as ox
# G = ox.graph_from_bbox(34.2946,34.2739,-118.1399,-118.0957,network_type="all_private",truncate_by_edge=False)
G = ox.graph_from_bbox(32.93053,32.92772,-117.17425,-117.17807,network_type="all_private",truncate_by_edge=True)
G = ox.utils_graph.get_undirected(G)
nx.draw(G,node_size=50)


In [6]:

# min_lat = 34.2739
# max_lat = 34.2946
# min_long = -118.0957
# max_long = -118.1399

min_lat = 32.92772
max_lat = 32.93053
min_long = -117.17425
max_long = -117.17807

new_network = nx.Graph()
added = {}
input = G
i = 0
for u,v,data in input.edges(keys=False,data=True):
    if("geometry" not in data):
        print("here")
        continue
    i += 1
    if u not in added:
        lat = input.nodes[u]['y']
        long = input.nodes[u]['x']
        if lat < min_lat or lat > max_lat:
            continue
        if abs(long) < abs(min_long) or abs(long) > abs(max_long):
            continue
        n1 = GSOM.Node(np.array([lat,long], dtype=float))
        new_network.add_node(n1)
        added[u] = n1
    if v not in added:
        lat = input.nodes[v]['y']
        long = input.nodes[v]['x']
        if lat < min_lat or lat > max_lat:
            continue
        if abs(long) < abs(min_long) or abs(long) > abs(max_long):
            continue
        n2 = GSOM.Node(np.array([lat,long], dtype=float))
        new_network.add_node(n2)
        added[v] = n2

    polyline = list(data["geometry"].coords)

    # Construct new node and edge
    weights = np.array([polyline[0][1],polyline[0][0]],dtype=float)
    prev_node = GSOM.Node(weights)
    new_network.add_node(prev_node)

    # new_network.add_edge(added[v],prev_node,weight=1)


    for pair in list(data["geometry"].coords[1:]):
        weights = np.array([pair[1],pair[0]],dtype=float)
        new_node = GSOM.Node(weights)
        new_network.add_node(new_node)
        new_network.add_edge(new_node,prev_node,weight=1)  
        prev_node = new_node
    

    # new_network.add_edge(prev_node,added[u],weight=1)  


In [7]:
gt_segments = GSOM.findSegments(new_network)
c_segments = GSOM.findSegments(basis.network)

amount of segments 5
amount of segments 8


In [14]:

pos_dict = {}
for node in new_network:
    pos_dict[node] = node.array
edges,weights = zip(*nx.get_edge_attributes(new_network,"weight").items())
nx.draw(new_network,pos=pos_dict,node_color='purple',node_size=50,edge_color='purple')

plt.show()

In [9]:
def lineDistance(point,polyline):
    minDistance = float("inf")
    for node in polyline:
        distance = GSOM.gpsDistance(point.array,node.array)
        if distance < minDistance:
            minDistance = distance
    return minDistance

seg_dict = {}
match_dict = {}
for id_c,edge_c in enumerate(c_segments):
    minDistance = float("inf")
    for id_t,edge_t in enumerate(gt_segments):
        n = 0
        distanceSum = 0
        for point_c in edge_c:
            distanceSum += lineDistance(point_c,edge_t)
            n += 1
        segmentDistance = distanceSum / n
        if segmentDistance < minDistance:
            minDistance = segmentDistance
            seg_dict[id_c] = id_t


matched_length = 0
print(seg_dict)
for x in seg_dict:
    segment = gt_segments[seg_dict[x]]
    for first,second in zip(segment,segment[1:]):
        matched_length += GSOM.gpsDistance(first.array,second.array)
print(matched_length)

total_length = 0
for segment in gt_segments:
    for first,second in zip(segment,segment[1:]):
        total_length += GSOM.gpsDistance(first.array,second.array)
print(total_length)
print(f"completeness {matched_length / total_length}")

{0: 2, 1: 1, 2: 2, 3: 4, 4: 2, 5: 4, 6: 3, 7: 2}
0.01570540278323437
0.007674215733130897
completeness 2.046515673964112


In [10]:

# paths = dict(nx.all_pairs_shortest_path(new_network))
# segments = []
# count = 0
# nodes_j = []
# for node,degree in list(new_network.degree()):
#     if degree == 1 or degree > 2:
#         nodes_j.append(node)



# for x1,x2 in itertools.combinations(nodes_j,2):
#     flag = True

#     if nx.has_path(new_network,x1,x2):
#         for y in paths[x1][x2][1:-1]:
#             for z in nodes_j:
#                 if z == y:
#                     flag = False
#                     break
#         if flag:
#             segments.append(paths[x1][x2])
#         count += 1
    
# print(f"amount of segments {len(segments)}")

# paths = dict(nx.all_pairs_shortest_path(basis.network))
# c_segments = []
# count = 0
# nodes_j = []
# for node,degree in list(basis.network.degree()):
#     if degree == 1 or degree > 2:
#         nodes_j.append(node)


# for x1,x2 in itertools.combinations(nodes_j,2):
#     flag = True
#     if nx.has_path(basis.network,x1,x2):
#         for y in paths[x1][x2][1:-1]:
#             for z in nodes_j:
#                 if z == y:
#                     flag = False
#                     break
#         if flag:
#             c_segments.append(paths[x1][x2])
#         count += 1
    
# print(len(c_segments))